# Answering Business Questions Using SQL

We will analyze a modified version of a database called Chinook.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Here's a schema diagram for the Chinook database:

![chinook-schema.svg](other/chinook-schema.svg)

We will try to answer the following questions:
- which genres sell the best in the USA,
- is any sales support agent performing either better or worse than the others,
- how business is performing at a country level,
- what percentage of purchases are individual tracks vs whole albums.

### 1. Loading the chinook database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### 2. Investigating the database

In [2]:
%%sql
SELECT
    name,
    type
  FROM sqlite_master
 WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


###  3. Finding out which genres sell the best in the USA

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce.

Artist Name	         | Genre
---------------------|------------
Regal	             | Hip-Hop
Red Tone             | Punk
Meteor and the Girls | Pop
Slim Jim Bites	     | Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [3]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* 
      FROM invoice_line AS il
     INNER JOIN invoice AS i 
        ON il.invoice_id = i.invoice_id
     INNER JOIN customer AS c 
        ON i.customer_id = c.customer_id
     WHERE c.country = "USA"
   )

SELECT
    g.name AS genre,
    COUNT(uts.invoice_line_id) AS tracks_sold,
    ROUND(CAST(COUNT(uts.invoice_line_id) AS FLOAT) / (
                                                 SELECT COUNT(*) 
                                                   FROM usa_tracks_sold
                                                 )*100, 1) AS percentage_sold
  FROM usa_tracks_sold AS uts
 INNER JOIN track AS t 
    ON t.track_id = uts.track_id
 INNER JOIN genre AS g 
    ON g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## 4. Analyzing performance of sales support agents

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [4]:
%%sql
wITH
    all_tables AS
                 (
                  SELECT
                      e.*,
                      i.*
                    FROM employee AS e
                   INNER JOIN customer AS c
                      ON e.employee_id = c.support_rep_id
                   INNER JOIN invoice AS i
                      ON c.customer_id = i.customer_id
                    WHERE e.title = 'Sales Support Agent'
                 )
SELECT
    at.first_name || ' ' || at.last_name AS employee_name,
    at.hire_date,
    ROUND(SUM(at.total), 2) AS total_sales
  FROM all_tables AS at
 GROUP BY at.employee_id
 ORDER BY employee_name
 LIMIT 10;
    



 * sqlite:///chinook.db
Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## 5.  Sales data for customers from each different country 

The next task is to analyze the sales data for customers from each different country (country value from the customers table). 

We will calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer
- customer lifetime value

Because there are a number of countries with only one customer, we will group these customers as "Other" in analysis.

In [5]:
%%sql
WITH
    tabel AS
            (
            SELECT
                CASE
                    WHEN (SELECT
                               COUNT(*)
                            FROM customer
                           WHERE country = c.country
                          ) = 1 THEN 'Other'
                    ELSE c.country
                END AS country_2,
                c.customer_id,
                i.total
              FROM customer AS c
              LEFT JOIN invoice AS i
                ON c.customer_id = i.customer_id
            )   
SELECT
    country,
    total_customers,
    total_sales,
    avg_sales,
    customer_lifetime_value
  FROM(
      SELECT
          country_2 AS country,
          COUNT(DISTINCT customer_id) AS total_customers,
          ROUND(SUM(total), 2) AS total_sales,
          ROUND(AVG(total), 2) AS avg_sales,
          ROUND(SUM(total) / count(distinct customer_id), 2) AS customer_lifetime_value,
          CASE
              WHEN country_2 = "Other" THEN 1
              ELSE 0
          END AS sort  
       FROM tabel
      GROUP BY country_2
      ORDER BY sort ASC, total_sales DESC
  );
    


 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_sales,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.57
Other,15,1094.94,7.45,73.0


Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## 6. Purchases of individual tracks vs whole albums

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [6]:
%%sql

WITH
    connection AS
               (
                SELECT
                    il.invoice_id AS invoice_id,
                    MIN(il.track_id) AS first_track_id
                  FROM invoice_line AS il
                 GROUP BY il.invoice_id
               ),
    category AS
               (
                SELECT
                    c.invoice_id,
                    CASE
                    WHEN (
                           SELECT
                               il2.track_id
                             FROM invoice_line AS il2
                             WHERE il2.invoice_id = c.invoice_id
                        
                           EXCEPT
                        
                           SELECT
                               t2.track_id
                             FROM track AS t2
                             WHERE t2.album_id = (SELECT
                                                     t3.album_id
                                                   FROM track AS t3
                                                   WHERE t3.track_id = c.first_track_id
                                                 )       
                           ) IS NULL
                         AND
                           (
                           SELECT
                               t4.track_id
                             FROM track AS t4
                            WHERE t4.album_id = (SELECT
                                                     t5.album_id
                                                   FROM track AS t5
                                                   WHERE t5.track_id = c.first_track_id
                                                )
                           EXCEPT
                                                
                           SELECT
                               il2.track_id
                             FROM invoice_line AS il2
                            WHERE il2.invoice_id = c.invoice_id
                           ) IS NULL 
                    THEN 'whole'
                    ELSE 'individual'
                    END AS 'cat'
                FROM connection AS c  
               ),
    whole AS
               (SELECT
                    cat AS category,
                    COUNT(cat) AS no_invoices,
                    ROUND(CAST(COUNT(cat) AS Float)/ (
                                             SELECT
                                                 COUNT(*)
                                                FROM category 
                    )*100, 1) AS pct_invoices
                  FROM category
                 WHERE cat = 'whole'
               ),
    individual AS
               (
                SELECT
                    cat AS category,
                    COUNT(cat) AS no_invoices,
                    ROUND(CAST(COUNT(cat) AS Float)/ (
                                             SELECT
                                                 COUNT(*)
                                                FROM category 
                    )*100, 1) AS pct_invoices
                  FROM category
                 WHERE cat = 'individual'   
               )
        
SELECT *
  FROM whole
    
UNION

SELECT *
  FROM individual

 * sqlite:///chinook.db
Done.


category,no_invoices,pct_invoices
individual,500,81.4
whole,114,18.6


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.